<a href="https://colab.research.google.com/github/DhruvMakwana/Global-Wheat-Detection/blob/master/Training_YOLOV4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training YOLOv4 in google colab GPU enabled

**Note**: Make sure you have your runtime type is GPU.

# Step 1: Cloning and Building Darknet

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (build)
%%time
!make

# Step 2: Download pretrained YOLOv4 weights

In [ ]:
# get yolov4 pretrained coco dataset weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline
 
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)
 
  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
 
# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
 
# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

# Step 3: Configure files for training

In [ ]:
!ln -s /content/drive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# download cfg to google drive and change required variables
!cp /content/darknet/cfg/yolov4.cfg /mydrive/Dataset/yolov4.cfg

In [ ]:
# Use this if you want to download file for local machine
#download("/content/darknet/cfg/yolov4.cfg")

Open yolov4.cfg in Text editor from drive and make some changes.
1. On Line 2: change batch = 16
2. On Line 3: change subdivision = 64
3. On Line 7: change width = 928
4. On Line 8: change height = 928
5. On Line 19: change max_batches = 6000
6. On Line 21: change steps = 2000, 3000
7. On Line 961: change filters = 18
8. On Line 967: change anchors to anchors we generate in [this notebook](https://github.com/DhruvMakwana/Global-Wheat-Detection/blob/master/Generate_Anchors_for_YOLO.ipynb) i.e. (1.30,2.49, 1.70,1.23, 2.21,2.01, 2.46,2.94, 2.82,4.55, 3.17,1.38, 3.72,2.28, 4.65,3.33, 5.55,5.97)
9. On Line 968: change classes = 1
10. On Line 1049: change filters = 18
11. On Line 1055: change anchors to anchors we generate in [this notebook](https://github.com/DhruvMakwana/Global-Wheat-Detection/blob/master/Generate_Anchors_for_YOLO.ipynb) i.e. (1.30,2.49, 1.70,1.23, 2.21,2.01, 2.46,2.94, 2.82,4.55, 3.17,1.38, 3.72,2.28, 4.65,3.33, 5.55,5.97)
12. On Line 1056: change classes = 1
13. On Line 1137: change filters = 18
14. On Line 1143: change anchors to anchors we generate in [this notebook](https://github.com/DhruvMakwana/Global-Wheat-Detection/blob/master/Generate_Anchors_for_YOLO.ipynb) i.e. (1.30,2.49, 1.70,1.23, 2.21,2.01, 2.46,2.94, 2.82,4.55, 3.17,1.38, 3.72,2.28, 4.65,3.33, 5.55,5.97)
15. On Line 1144: change classes = 1

In [ ]:
# upload the  yolov4.cfg and train.txt back to colab from Google Drive
!cp /mydrive/Dataset/yolov4.cfg /content/darknet/cfg
!cp /mydrive/train.txt /content/darknet/data
 
# upload the yolov4.cfg and train.txt back to colab from local machine (uncomment to use)
#%cd cfg
#upload()
#%cd ..

Create two new files in google drive in Dataset folder named `obj.names` and `obj.data`

1. obj.names: Write this 1 line in obj.names file
    ```
    Wheat
    ```
2. obj.data: Write this 4 lines in obj.data file

    ```
    classes = 1
    train = /content/darknet/data/train.txt
    names = /content/darknet/data/obj.names
    backup = "/content/drive/My Drive/Dataset/Backup/"
    ```

In [ ]:
# upload the obj.names and obj.data files to colab from Google Drive
!cp /mydrive/Dataset/obj.names /content/darknet/data/
!cp /mydrive/Dataset/obj.data /content/darknet/data/
 
# upload the obj.names and obj.data files to colab from local machine (uncomment to use)
#%cd data
#upload()
#%cd ..

# Step 4: Download pre-trained weights for convolution layers

In [ ]:
# download pretrained convolutional layer weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# Step 5: Train Object Detector!

In [ ]:
!./darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/yolov4.cfg /content/darknet/yolov4.conv.137 -dont_show

In [ ]:
# if training stops before completion then run this cell it will continue from last saved result
!./darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/yolov4.cfg /mydrive/Dataset/Backup/yolov4_last.weights -dont_show